# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
J. Li  ->  J. Li  |  ['J. Li']
J. Li  ->  J. Li  |  ['J. Li']


J. Li  ->  J. Li  |  ['J. Li']
Arxiv has 75 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2410.01874


extracting tarball to tmp_2410.01874...

 done.


Found 245 bibliographic references in tmp_2410.01874/main.bbl.
Issues with the citations
syntax error in line 1278: '=' expected
Retrieving document from  https://arxiv.org/e-print/2410.01884


extracting tarball to tmp_2410.01884...

 done.
Retrieving document from  https://arxiv.org/e-print/2410.01886
extracting tarball to tmp_2410.01886...

 done.
Retrieving document from  https://arxiv.org/e-print/2410.02731


extracting tarball to tmp_2410.02731...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.01874-b31b1b.svg)](https://arxiv.org/abs/2410.01874) | **All the Little Things in Abell 2744: $>$1000 Gravitationally Lensed Dwarf Galaxies at $z=0-9$ from JWST NIRCam Grism Spectroscopy**  |
|| R. P. Naidu, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2024-10-04*|
|*Comments*| *Submitted to the Open Journal of Astrophysics. Figs. 6 and 10 illustrate the quality of the spectra and imaging, while Fig. 12 summarizes the yield of the survey. Comments warmly welcomed and greatly appreciated. The ALT DR1 catalog is available at this https URL*|
|**Abstract**|            Dwarf galaxies hold the key to crucial frontiers of astrophysics, however, their faintness renders spectroscopy challenging. Here we present the JWST Cycle 2 survey, All the Little Things (ALT, PID 3516), which is designed to seek late-forming Pop III stars and the drivers of reionization at $z\sim6-7$. ALT has acquired the deepest NIRCam grism spectroscopy yet (7-27 hr), at JWST's most sensitive wavelengths (3-4 $\mu$m), covering the powerful lensing cluster Abell 2744. Over the same 30 arcmin$^2$, ALT's ultra-deep F070W+F090W imaging ($\sim$30 mag) enables selection of very faint sources at $z>6$. We demonstrate the success of ALT's novel ``butterfly" mosaic to solve spectral confusion and contamination, and introduce the ``Allegro" method for emission line identification. By collecting spectra for every source in the field of view, ALT has measured precise ($R\sim1600$) redshifts for 1630 sources at $z=0.2-8.5$. This includes one of the largest samples of distant dwarf galaxies: [1015, 475, 50] sources less massive than the SMC, Fornax, and Sculptor with $\log(M_{*}/M_{\odot})<$[8.5, 7.5, 6.5]. We showcase ALT's discovery space with: (i) spatially resolved spectra of lensed clumps in galaxies as faint as $M_{\rm{UV}}\sim-15$; (ii) large-scale clustering -- overdensities at $z$=[2.50, 2.58, 3.97, 4.30, 5.66, 5.77, 6.33] hosting massive galaxies with striking Balmer breaks; (iii) small-scale clustering -- a system of satellites around a Milky Way analog at $z\sim6$; (iv) spectroscopically confirmed multiple images that help constrain the lensing model underlying all science in this legacy field; (v) sensitive star-formation maps based on dust-insensitive tracers such as Pa$\alpha$; (vi) direct spectroscopic discovery of rare sources such as AGN with ionized outflows. These results provide a powerful proof of concept for how grism surveys maximize the potential of strong lensing fields.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.01884-b31b1b.svg)](https://arxiv.org/abs/2410.01884) | **Merian: A Wide-Field Imaging Survey of Dwarf Galaxies at z~0.06-0.10**  |
|| S. Danieli, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2024-10-04*|
|*Comments*| *24 pages, 13 figures, 2 tables. Submitted to ApJ*|
|**Abstract**|            We present the Merian Survey, an optical imaging survey optimized for studying the physical properties of bright star-forming dwarf galaxies. Merian is carried out with two medium-band filters ($N708$ and $N540$, centered at $708$ and $540$ nm), custom-built for the Dark Energy Camera (DECam) on the Blanco telescope. Merian covers $\sim 750\,\mathrm{deg}^2$ of equatorial fields, overlapping with the Hyper Suprime-Cam Subaru Strategic Program (HSC-SSP) wide, deep, and ultra-deep fields. When combined with the HSC-SSP imaging data ($grizy$), the new Merian DECam medium-band imaging allows for photometric redshift measurements via the detection of H$\rm\alpha$ and [OIII] line emission flux excess in the $N708$ and $N540$ filters, respectively, at $0.06<z<0.10$. We present an overview of the survey design, observations taken to date, data reduction using the LSST Science Pipelines, including aperture-matched photometry for accurate galaxy colors, and a description of the data included in the first data release (DR1). The key science goals of Merian include: probing the dark matter halos of dwarf galaxies out to their virial radii using high signal-to-noise weak lensing profile measurements, decoupling the effects of baryonic processes from dark matter, and understanding the role of black holes in dwarf galaxy evolution. This rich dataset will also offer unique opportunities for studying extremely metal-poor galaxies via their strong [OIII] emission and H$\rm\alpha$ lines, as well as [OIII] emitters at $z\sim 0.4$, and Ly$\rm\alpha$ emitters at $z\sim 3.3$ and $z\sim 4.8$. Merian showcases the power of utilizing narrow and medium-band filters alongside broad-band filters for sky imaging, demonstrating their synergistic capacity to unveil astrophysical insights across diverse astrophysical phenomena.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.01886-b31b1b.svg)](https://arxiv.org/abs/2410.01886) | **A Nonparametric Morphological Analysis of H$\alpha$ Emission in Bright Dwarfs Using the Merian Survey**  |
|| A. Mintz, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2024-10-04*|
|*Comments*| *33 pages, 15 figures, 2 tables. Accepted to ApJ*|
|**Abstract**|            Using medium-band imaging from the newly released Merian Survey, we conduct a nonparametric morphological analysis of H$\alpha$ emission maps and stellar continua for a sample of galaxies with $8\lesssim\log (M_\star/M_\odot) < 10.3$ at $0.064<z<0.1$. We present a novel method for estimating the stellar continuum emission through the Merian Survey's N708 medium-band filter, which we use to measure H$\alpha$ emission and produce H$\alpha$ maps for our sample of galaxies with seven-band Merian photometry and available spectroscopy. We measure nonparametric morphological statistics for the H$\alpha$ and stellar continuum images, explore how the morphology of the H$\alpha$ differs from the continuum, and investigate how the parameters evolve with the galaxies' physical properties. In agreement with previous results for more massive galaxies, we find that the asymmetry of the stellar continuum increases with specific star formation rate (SSFR) and we extend the trend to lower masses, also showing that it holds for the asymmetry of the H$\alpha$ emission. We find that the lowest-mass galaxies with the highest SSFR have H$\alpha$ emission that is consistently heterogeneous and compact, while the less active galaxies in this mass range have H$\alpha$ emission that appears diffuse. At higher masses, our data do not span a sufficient range in SSFR to evaluate whether similar trends apply. We conclude that high SSFRs in low-mass galaxies likely result from dynamical instabilities that compress a galaxy's molecular gas to a dense region near the center.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.02731-b31b1b.svg)](https://arxiv.org/abs/2410.02731) | **Extinction of Taurus, Orion, Perseus and California Molecular Clouds Based on the LAMOST, 2MASS, and Gaia Surveys II: The Extinction Law**  |
|| Z. Cao, B. Jiang, S. Wang, <mark>J. Li</mark> |
|*Appeared on*| *2024-10-04*|
|*Comments*| *9 pages, 10 figures, Accepted by The Astronomical Journal*|
|**Abstract**|            The extinction law from ultraviolet (UV) to infrared (IR) (0.2-24 $\mu$m) is determined by relying on the blue-edge method and color excess ratios for some nearby molecular clouds, from low mass star forming region to massive star forming region. The observational data are collected from nine photometric surveys, along with stellar parameters from the APOGEE and LAMOST spectroscopic surveys. Within the uncertainties, the optical ratio of selective to total extinction ($R_{G_{BP}}$) does not vary substantially across the clouds, irrespective of the density, specifically $R_{G_{BP}} =2.302\pm0.027$, where $R_{G_{BP}} \equiv A_{G_{BP}}/E_{G_{BP},G_{RP}}$. The IR extinction law is consistent with \citet{Wang19_law}. The extinction law in the UV band is compromised by the shallow depth with $A_{V} \leq 2$ mag and is hard to describe by one parameter $R$. In addition, the extinction in the WISE/W1 band is significantly larger than in the Spitzer/IRAC1 band in the dense regions, which is attributed to the ice water absorption.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2410.01874/./figs/ALT_roll_example.png', 'tmp_2410.01874/./figs/ALT_field_coverage.png', 'tmp_2410.01874/./figs/redshift_distribution.png']
copying  tmp_2410.01874/./figs/ALT_roll_example.png to _build/html/
copying  tmp_2410.01874/./figs/ALT_field_coverage.png to _build/html/
copying  tmp_2410.01874/./figs/redshift_distribution.png to _build/html/
exported in  _build/html/2410.01874.md
    + _build/html/tmp_2410.01874/./figs/ALT_roll_example.png
    + _build/html/tmp_2410.01874/./figs/ALT_field_coverage.png
    + _build/html/tmp_2410.01874/./figs/redshift_distribution.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\xiion}{\xi_{\rm{ion}}}$
$\newcommand{\halpha}{H\ensuremath{\alpha}}$
$\newcommand{\hbeta}{H\ensuremath{\beta}}$
$\newcommand{\mstar}{\ensuremath{\log(M_{\rm{\star}}/M_{\rm{\odot}})}}$
$\newcommand{\orcidauthor}[3]{\author{\href{http://orcid.org/#1}{#2^{#3}}}}$</div>



<div id="title">

# $\vspace{-0.8cm}$All the Little Things in Abell 2744: $>1000$ Gravitationally Lensed Dwarf Galaxies at $\MakeLowercase{z}=0-9$ from JWST NIRCam Grism Spectroscopy$\vspace{-1.6cm}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2410.01874-b31b1b.svg)](https://arxiv.org/abs/2410.01874)<mark>Appeared on: 2024-10-04</mark> -  _Submitted to the Open Journal of Astrophysics. Figs. 6 and 10 illustrate the quality of the spectra and imaging, while Fig. 12 summarizes the yield of the survey. Comments warmly welcomed and greatly appreciated. The ALT DR1 catalog is available at this https URL_

</div>
<div id="authors">

R. P. Naidu, et al. -- incl., <mark>A. d. Graaff</mark>

</div>
<div id="abstract">

**Abstract:** Dwarf galaxies hold the key to crucial frontiers of astrophysics, however, their faintness renders spectroscopy challenging. Here we present the JWST Cycle 2 survey, All the Little Things (ALT, PID 3516), which is designed to seek late-forming Pop III stars and the drivers of reionization at $z\sim6$ - $7$ . ALT has acquired the deepest NIRCam grism spectroscopy yet (7-27 hr), at JWST's most sensitive wavelengths (3-4 $\mu$ m), covering the powerful lensing cluster Abell 2744. Over the same 30 arcmin $^{2}$ , ALT's ultra-deep F070W+F090W imaging ( $\sim30$ mag, 5 $\sigma$ ) enables selection of very faint sources at $z>6$ . We demonstrate the success of ALT's novel "butterfly" mosaic to solve spectral confusion and contamination, and introduce the "Allegro" method for emission line identification. By collecting spectra for every source in the field of view, ALT has measured precise ( $\mathcal{R}\sim1600$ ) redshifts for 1630 sources at $z=0.2$ - $8.5$ . This includes one of the largest samples of distant dwarf galaxies: [ 1015, 475, 50 ] sources less massive than the SMC, Fornax, and Sculptor with $\log(M_{*}/M_{\rm{\odot}})<$ [ 8.5, 7.5, 6.5 ] . We showcase ALT's discovery space with: (i) spatially resolved spectra of lensed clumps in galaxies as faint as $M_{\rm{UV}}\sim-15$ ; (ii) large-scale clustering -- overdensities at $z$ = [ 2.50, 2.58, 3.97, 4.30, 5.66, 5.77, 6.33 ] hosting massive galaxies with striking Balmer breaks; (iii) small-scale clustering -- a system of satellites around a Milky Way analog at $z\sim6$ ; (iv) spectroscopically confirmed multiple images that help constrain the lensing model underlying all science in this legacy field; (v) sensitive star-formation maps based on dust-insensitive tracers such as Pa $\alpha$ ; (vi) direct spectroscopic discovery of rare sources such as AGN with ionized outflows. These results provide a powerful proof of concept for how grism surveys maximize the potential of strong lensing fields.

</div>

<div id="div_fig1">

<img src="tmp_2410.01874/./figs/ALT_roll_example.png" alt="Fig4" width="100%"/>

**Figure 4. -** **Illustration of how the mosaic design facilitates the analysis of the grism data**. **Left**: ALT field of view in the two rolls (solid and dashed black line) on top of the same background F356W image as in Figure $\ref{fig:RADEC}$. Blue circles show the galaxies for which the ALT data measured a spectroscopic redshift. The three shaded regions highlight the stripes observed twice (either twice with module A, or once with each module, or twice with module B, from left to right respectively). We highlight the positions of four example objects with red squares. **Right:** NIRCam false colour stamps (based on F115W/F200W/F356W imaging) and 2D grism emission-line spectra for the four objects highlighted in the left panel. The grism spectra span $\approx15$ nm in the observed frame. For objects 1 and 4, we show 2D spectra separated by Roll 1 vs. Roll 2, while for objects 2 and 3 we separate Module A vs. Module B. These spectra illustrate how observations of the same source across different rolls and modules helps in disentangling contamination and confusion. (*fig:rolls*)

</div>
<div id="div_fig2">

<img src="tmp_2410.01874/./figs/ALT_field_coverage.png" alt="Fig3" width="100%"/>

**Figure 3. -** **The ** The ALT survey footprint around the Abell 2744 lensing cluster**.** Top: Maximum on sky field of view of the ALT NIRCam WFSS data (red lines; at the undeviated wavelength, $\lambda=3.95 \mu$m) illustrated on top of a background F356W image (combining ALT and UNCOVER data). The dashed purple line shows the minimum field of view at 3.15 $\mu$m. Contours highlight the increasing magnification closer to the three main cores of the cluster (at $z=6$ from the model by \citealt{Furtak23}). The center of the footprint is at $\alpha=$0:14:18.0, $\delta=-$30:22:47.3 (J2000). Bottom, left: Wavelength dependence of the spectroscopic data and the total field of view (along the main axis of the mosaic, it is constant along the minor axis). Our data are most sensitive in the deep stripes and around 3.8 $\mu$m. Bottom, right: Spatial variation of the sensitivity of the grism data, shown at a wavelength of 3.8 $\mu$m. The deep stripes and mosaic pattern, as well as the difference in sensitivity between module A and B data can clearly be seen. One can also notice the slight decrease in sensitivity near bright objects. (*fig:RADEC*)

</div>
<div id="div_fig3">

<img src="tmp_2410.01874/./figs/redshift_distribution.png" alt="Fig1" width="100%"/>

**Figure 1. -** ** Spectroscopic redshift distribution of the emission-line galaxies in ALT DR1.** The red histogram shows all redshifts, while the blue histogram show the galaxies for which more than one line is detected above S/N $>10$. This illustrates that the majority of pairs are HeI+Pa$\gamma$ at $z\sim2.5$ and the [OIII] doublet at $z\sim6$ and that the vast majority of H$\alpha$ emitters are single-line emitters in our data. We highlight the redshift ranges at which the primary lines are covered in the F356W grism spectra.  (*fig:redshifts*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2410.01874"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

206  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

9  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
